In [27]:
from IPython.display import display, HTML,clear_output
import random as rnd
import math
import time

In [28]:
class SVGCanvas:
    def __init__(self,width,height,size):
        self.width = width
        self.height = height
        self.size = size
        self.html_s =""
    
    def clear(self):
        self.html_s = '<svg width="'+str(self.width)+'" height="'+str(self.height)+'"style="border:1px solid black">'

    def addCircle(self,x_pos,y_pos,color):
        x_pos = x_pos/self.size
        y_pos = y_pos/self.size
        r = 1/(self.size*2)
        str_int_x = str(int((x_pos*self.width)+(r*self.width)))
        str_int_y = str(int((y_pos*self.height)+(r*self.width)))
        str_int_r = str(int(r*self.width))
        self.html_s +='<circle cx="'+str_int_x+'" cy="'+str_int_y+'" r="'+str_int_r+'" fill="'+color+'"/>'

    def addRect(self,x_pos,y_pos,color):
        x_pos = x_pos/self.size
        y_pos = y_pos/self.size
        height = 1/self.size
        width = 1/self.size
        str_int_x = str(int(x_pos*self.width))
        str_int_y = str(int(y_pos*self.height))
        str_int_width = str(int(width*self.width))
        str_int_height = str(int(height*self.height))
        self.html_s +='<rect x="'+str_int_x+'" y="'+str_int_y+'" width="'+str_int_width+'" height="'+str_int_height+'" fill="'+color+'"/>'

    def getCanvas(self):
        self.html_s += "</svg>"
        return HTML(self.html_s)


In [29]:
class Cell:
    def __init__(self,x_pos,y_pos,resource):
        self.occupant = None
        self.x_pos = x_pos
        self.y_pos = y_pos
        self.resource = resource
        
    def update(self):
        self.resource = rnd.random()
        

In [37]:
class Agent:
    world=None
    
    def __init__(self,x_pos,y_pos,distance):
        self.x_pos=x_pos
        self.y_pos=y_pos
        self.distance = distance
        Agent.world.grid[x_pos][y_pos].occupant = self
        
    def move(self,dx,dy):
        cells = []
        
        for ax in range(-self.distance,self.distance+1):
            dx = Agent.world.bounds(self.x_pos+ax)
            cells.append(Agent.world.grid[dx][self.y_pos])
        for ay in range(-self.distance,self.distance+1):
            dy = Agent.world.bounds(self.y_pos+ay)
            cells.append(Agent.world.grid[self.x_pos][dy])
        rnd.shuffle(cells)
        new_pos=None
        max_res = 0
        for c in cells:
            if c.resource>max_res:
                max_res = c.resource
                new_pos = c
        new_x_pos=new_pos.x_pos
        new_y_pos=new_pos.y_pos
        if Agent.world.grid[new_x_pos][new_y_pos].occupant==None:
            Agent.world.grid[self.x_pos][self.y_pos].occupant=None
            self.x_pos = new_x_pos
            self.y_pos = new_y_pos
            Agent.world.grid[self.x_pos][self.y_pos].occupant = self
            Agent.world.grid[self.x_pos][self.y_pos].resource=0
        

In [38]:
class World:
    def __init__(self,size,number_of_agents):
        h_size = int(size/2)
        self.size =size
        self.grid=[]
        self.svg = SVGCanvas(601,601,size)
        for x in range(size):
            self.grid.append([])
            for y in range(size):
                dx = (h_size-abs(x-h_size))/h_size*0.5
                dy = (h_size-abs(y-h_size))/h_size*0.5
                self.grid[x].append(Cell(x,y,dx+dy))
        Agent.world = self
        
        self.agents=[]
        for i in range(number_of_agents):
            flag = True
            while flag:
                x_pos = rnd.randint(0,self.size-1)
                y_pos = rnd.randint(0,self.size-1)
                if self.grid[x_pos][y_pos].occupant==None:
                    self.agents.append(Agent(x_pos,y_pos,rnd.randint(1,3)))
                    flag = False
                                       
    def bounds(self,pos):
        if pos<0:
            return self.size+pos
        if pos>= self.size:
            return pos-self.size
        return  pos
                                    
    def draw(self):
        colors=["","rgb(255,0,0)","rgb(0,255,0)","rgb(0,0,255)","rgb(255,0,255)"]
        self.svg.clear()
        for i in range(self.size):
            for j in range(self.size):
                b = str(255- int(self.grid[i][j].resource*255))
                self.svg.addRect(i,j,'rgb(255,255,'+b+')')
                if self.grid[i][j].occupant !=None:
                    a = self.grid[i][j].occupant
                    self.svg.addCircle(i,j,colors[a.distance])
        display(self.svg.getCanvas())
                                 
    def update(self):
        for a in self.agents:
            a.move(rnd.randint(-1,1),rnd.randint(-1,1))
    
    def run(self,run_time): 
        for t in range(run_time):
            clear_output(wait=True)
            self.update()
            self.draw()                            
            time.sleep(0.01)
        

In [39]:
world = World(62,10)

In [40]:
world.run(40)
print("done")

AttributeError: 'NoneType' object has no attribute 'x_pos'